# AAR1

In [ ]:
import pandas as pd
import random
import math

# ---------------------------------------
# Step 1: Load and Prepare Input Data
# ---------------------------------------

# excel_path: String path to the input Excel file (AA-R1.xlsx) containing all required sheets
excel_path = '/content/AA-R1.xlsx'  # Path can be adjusted based on file location

# xls: ExcelFile object to efficiently load multiple sheets from the Excel file
xls = pd.ExcelFile(excel_path)

# ---------------------------------------
# 1.1 Load and Process Renovation Blocks Sheet
# ---------------------------------------
# renovation_block_data: DataFrame containing renovation block data from 'RenovationProgram Table Input 1' sheet
renovation_block_data = xls.parse('RenovationProgram Table Input 1')
# Clean column names by removing leading/trailing whitespace
renovation_block_data.columns = renovation_block_data.columns.str.strip()

# Rename columns for consistency and clarity in downstream processing
renovation_block_data = renovation_block_data.rename(columns={
    'Cumulative_Block_Circulation_Area': 'Cumulative_Area_SQM',  # Total area in square meters for each block
    'Max_Occupancy_with_Capacity': 'Max_Occupancy_with_Capacity'  # Maximum occupancy, already correctly named
})

# ---------------------------------------
# 1.2 Ensure Numeric Columns
# ---------------------------------------
# Convert key columns to numeric, handling non-numeric values gracefully
# existing_block_data: DataFrame assumed to exist (though not loaded in the original code, included for consistency)
# Note: existing_block_data is referenced but not defined; assuming it's similar to renovation_block_data
try:
    for df in [renovation_block_data, existing_block_data]:
        # Cumulative_Area_SQM: Total area in square meters, converted to numeric
        df['Cumulative_Area_SQM'] = pd.to_numeric(
            df['Cumulative_Area_SQM'], errors='coerce'  # Convert invalid entries to NaN
        ).fillna(0)  # Replace NaN with 0
        # Max_Occupancy_with_Capacity: Maximum occupancy, converted to numeric
        df['Max_Occupancy_with_Capacity'] = pd.to_numeric(
            df['Max_Occupancy_with_Capacity'], errors='coerce'  # Convert invalid entries to NaN
        ).fillna(0)  # Replace NaN with 0
except NameError:
    # If existing_block_data is not defined, process only renovation_block_data
    renovation_block_data['Cumulative_Area_SQM'] = pd.to_numeric(
        renovation_block_data['Cumulative_Area_SQM'], errors='coerce'
    ).fillna(0)
    renovation_block_data['Max_Occupancy_with_Capacity'] = pd.to_numeric(
        renovation_block_data['Max_Occupancy_with_Capacity'], errors='coerce'
    ).fillna(0)

# all_block_data: Combined DataFrame of all blocks (only renovation_block_data in this case)
all_block_data = pd.concat([renovation_block_data], ignore_index=True)

# ---------------------------------------
# 1.3 Load and Process Floors Sheet
# ---------------------------------------
# all_floor_data: DataFrame containing floor data from 'Program Table Input 2 - Floor' sheet
all_floor_data = xls.parse('Program Table Input 2 - Floor', skiprows=0)  # No header rows skipped
all_floor_data.columns = all_floor_data.columns.str.strip()  # Clean column names
print(all_floor_data.columns.tolist())  # Debug: Display column names for verification

# Rename columns for clarity
all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',  # Floor name or identifier
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',  # Usable floor area in square meters
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'  # Maximum capacity for the floor
})
print(all_floor_data.columns.tolist())  # Debug: Display renamed column names

# Convert floor area and capacity to numeric
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='coerce'  # Handle non-numeric values
).fillna(0)  # Replace NaN with 0
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='coerce'
).fillna(0)  # Replace NaN with 0

# ---------------------------------------
# 1.4 Load and Process Department Split Sheet
# ---------------------------------------
# department_split_data: DataFrame containing department split rules from 'Department Split' sheet
department_split_data = xls.parse('Department Split', header=1)  # Use second row as header
department_split_data.columns = department_split_data.columns.str.strip()  # Clean column names
print(department_split_data.columns.tolist())  # Debug: Display column names

# Rename columns based on their intended use
department_split_data = department_split_data.rename(columns={
    department_split_data.columns[0]: 'Department_Sub-Department',  # Department or sub-department identifier
    department_split_data.columns[1]: 'Splittable',  # Indicates if department can be split across floors
    department_split_data.columns[2]: 'Min_%of_Block_per_department'  # Minimum percentage of block per department
})
print(department_split_data.columns.tolist())  # Debug: Display renamed column names

# Create dictionaries for quick lookups
# dept_splittable: Dictionary mapping departments to their splittable status (e.g., 1 = not splittable, -1 = splittable)
dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
# dept_min_pct: Dictionary mapping departments to their minimum percentage requirement
dept_min_pct = department_split_data.set_index('Department_Sub-Department')['Min_%of_Block_per_department'].to_dict()

# ---------------------------------------
# 1.5 Load Min%Split Sheet (Unused)
# ---------------------------------------
# min_split_data: DataFrame containing minimum split data (loaded but not used in the script)
min_split_data = xls.parse('Min % Split')
min_split_data.columns = min_split_data.columns.str.strip()

# ---------------------------------------
# 1.6 Load and Process Adjacency Sheet
# ---------------------------------------
# adjacency_sheet_name: String name of the sheet containing adjacency data (dynamically identified)
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
# raw_adj: Raw DataFrame from adjacency sheet, with header and index
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
# adjacency_data: Processed DataFrame with numeric values for adjacency relationships
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()  # Clean index
adjacency_data.columns = adjacency_data.columns.str.strip()  # Clean columns
# adj_lookup: Dictionary for quick adjacency lookups (dept -> dept -> value)
adj_lookup = adjacency_data.to_dict()

# ---------------------------------------
# 1.7 Load and Process De-Centralized Logic Sheet
# ---------------------------------------
# df_logic: DataFrame containing decentralized logic rules
df_logic = xls.parse('De-Centralized Logic', header=None)
# De_Centralized_data: Dictionary storing centralized/semi/decentralized settings
De_Centralized_data = {}
# current_section: Tracks the current section (Centralised, Semi Centralized, DeCentralised)
current_section = None
for _, row in df_logic.iterrows():
    # first_cell: First column value in the current row
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ''
    if first_cell in ['Centralised', 'Semi Centralized', 'DeCentralised']:
        current_section = first_cell
        De_Centralized_data[current_section] = {'Add': 0}  # Initialize with 'Add' key
    elif current_section and 'Add' in str(first_cell):
        De_Centralized_data[current_section]['Add'] = int(row[1]) if pd.notna(row[1]) else 0
# Ensure all sections exist with default values
for key in ['Centralised', 'Semi Centralized', 'DeCentralised']:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {'Add': 0}

# ---------------------------------------
# Step 2: Preprocess Blocks
# ---------------------------------------

# immovable_blocks: DataFrame of blocks marked as 'Immovable Asset'
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'
].copy()
# movable_blocks: DataFrame of blocks marked as 'Movable Asset'
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Movable Asset'
].copy()

# destination_blocks: Subset of movable blocks marked as 'Destination'
destination_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Destination'
].copy()
# typical_blocks: Subset of movable blocks marked as 'Typical'
typical_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Typical'
].copy()

# ---------------------------------------
# Step 3: Initialize Floor Assignments
# ---------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Initializes a dictionary for tracking floor assignments.

    Args:
        floor_df: DataFrame containing floor data (Name, Usable_Area_(SQM), Max_Assignable_Floor_loading_Capacity)

    Returns:
        assignments: Dictionary with floor names as keys and assignment details as values
    """
    # assignments: Dictionary mapping floor names to their properties and assignments
    assignments = {}
    for _, row in floor_df.iterrows():
        # floor: String name of the floor
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],  # Remaining usable area in square meters
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],  # Remaining capacity
            'assigned_blocks': [],  # List of assigned block dictionaries
            'assigned_departments': set(),  # Set of assigned department names
            'DeptArea': {},  # Dictionary tracking area per sub-department
            'ME_area': 0.0,  # Area for ME (Medical) category
            'WE_area': 0.0,  # Area for WE (Work Environment) category
            'US_area': 0.0,  # Area for US (User Space) category
            'Support_area': 0.0,  # Area for Support category
            'Speciality_area': 0.0  # Area for Speciality category
        }
    return assignments

# floors: List of floor names
floors = list(initialize_floor_assignments(all_floor_data).keys())

# all_categories: List of space mix categories
all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']

# ---------------------------------------
# Helper Functions
# ---------------------------------------

def can_place_block(blk, fl, assignments, mode):
    """
    Checks if a block can be placed on a floor based on adjacency, decentralization, and split rules.

    Args:
        blk: Dictionary representing a block
        fl: String floor name
        assignments: Dictionary of current floor assignments
        mode: String indicating stacking mode ('centralized', 'semi', 'decentralized')

    Returns:
        Boolean indicating if the block can be placed
    """
    # dept: Department or sub-department name for the block
    dept = blk.get('Department_Sub_Department', '').strip()
    # Check adjacency constraints
    for other in assignments[fl]['assigned_departments']:
        if dept in adj_lookup and other in adj_lookup[dept] and adj_lookup[dept][other] == -1:
            return False  # Hard adjacency forbid
    # Check destination floor restrictions for decentralized mode
    if mode == 'decentralized' and blk.get('Typical_Destination', '') == 'Destination':
        # max_dest: Maximum number of floors for destination blocks
        max_dest = 2 + De_Centralized_data['DeCentralised']['Add']
        if fl not in floors[:max_dest]:
            return False
    # Check department split rules
    # spl: Splittable status for the department (-1 = splittable, 1 = not splittable, 0 = waterfall)
    spl = dept_splittable.get(dept, -1)
    if spl == 1:
        # Department must stay on one floor
        for f2 in assignments:
            if f2 != fl and dept in assignments[f2]['assigned_departments']:
                return False
    elif spl == 0:
        # Waterfall rule: enforce minimum percentage
        # min_pct: Minimum percentage of department area required on the floor
        min_pct = dept_min_pct.get(dept, 100) / 100.0
        # used: Total area used by the department on this floor
        used = assignments[fl]['DeptArea'].get(dept, 0.0) + blk.get('Cumulative_Area_SQM', 0)
        # floor_area: Total usable area of the floor
        floor_area = all_floor_data.loc[all_floor_data['Name'] == fl, 'Usable_Area_(SQM)'].iloc[0]
        if used / floor_area < min_pct:
            return False
    return True

def primary_category(blk):
    """
    Determines the primary space mix category for a block.

    Args:
        blk: Dictionary representing a block

    Returns:
        String category name (ME, WE, US, or Support)
    """
    # mix: Space mix category string (e.g., 'ME', 'WE')
    mix = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '')
    for cat in ['ME', 'WE', 'US']:
        if cat in mix:
            return cat
    return 'Support'  # Default category

# ---------------------------------------
# Step 4: Core Stacking Function
# ---------------------------------------

def run_stack_plan(mode):
    """
    Runs the stacking plan for a given mode (centralized, semi, decentralized).

    Args:
        mode: String indicating stacking mode ('centralized', 'semi', 'decentralized')

    Returns:
        Tuple of DataFrames: detailed assignments, floor summary, space mix, unassigned blocks
    """
    # assignments: Dictionary tracking floor assignments
    assignments = initialize_floor_assignments(all_floor_data)
    # unassigned_blocks: List of blocks that could not be assigned
    unassigned_blocks = []

    # ---------------------------------------
    # Phase 0: Assign Immovable Blocks
    # ---------------------------------------
    for _, blk_series in immovable_blocks.iterrows():
        # blk: Dictionary representation of a block
        blk = blk_series.to_dict()
        # assigned_level: Specified floor level for the block
        assigned_level = blk.get('Level', '').strip()
        # blk_area: Area of the block in square meters
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        # blk_capacity: Occupancy capacity of the block
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)

        # matching_floor: Floor name matching the block's assigned level
        matching_floor = None
        for floor_name in floors:
            if assigned_level.lower() in floor_name.lower():
                matching_floor = floor_name
                break

        if matching_floor and assignments[matching_floor]['remaining_area'] >= blk_area and assignments[matching_floor]['remaining_capacity'] >= blk_capacity:
            assignments[matching_floor]['assigned_blocks'].append(blk)
            assignments[matching_floor]['remaining_area'] -= blk_area
            assignments[matching_floor]['remaining_capacity'] -= blk_capacity
            assignments[matching_floor]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
            # cat: Space mix category of the block
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
            if cat == 'ME':
                assignments[matching_floor]['ME_area'] += blk_area
            elif cat == 'WE':
                assignments[matching_floor]['WE_area'] += blk_area
            elif cat == 'US':
                assignments[matching_floor]['US_area'] += blk_area
            elif cat.lower() == 'support':
                assignments[matching_floor]['Support_area'] += blk_area
            elif cat.lower() == 'speciality':
                assignments[matching_floor]['Speciality_area'] += blk_area
        else:
            unassigned_blocks.append(blk)

    # ---------------------------------------
    # Phase 1: Assign Destination Blocks
    # ---------------------------------------

    def destination_floor_count():
        """Calculates the number of floors for destination blocks based on mode."""
        if mode == Modelo == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    # max_dest_floors: Maximum number of floors for destination blocks
    max_dest_floors = min(destination_floor_count(), len(floors))

    # dest_groups: Dictionary grouping destination blocks by Destination_Group
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        # grp: Destination group identifier
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Assign destination groups
    # group_names: List of destination group identifiers
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        # info_grp: Information about the destination group
        info_grp = dest_groups[grp]
        # grp_area: Total area of the group
        grp_area = info_grp['total_area']
        # grp_cap: Total capacity of the group
        grp_cap = info_grp['total_capacity']
        placed_whole = False

        # Attempt to place entire group
        # candidate_floors: List of candidate floors for placement
        candidate_floors = floors[:max_dest_floors].copy()
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'])
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # Try remaining floors if not placed
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # Split group if necessary
        if not placed_whole:
            # total_remaining_area: Total remaining area across all floors
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # blocks_sorted: Blocks sorted by area (largest first)
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                # removed_blocks: Blocks removed during trial placement
                removed_blocks = []
                # trial_blocks: Blocks to attempt placing as a group
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    # trial_area: Total area of trial blocks
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    # trial_capacity: Total capacity of trial blocks
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)
                    # floor_combination: List of (block, floor) assignments
                    floor_combination = []
                    # temp_assignments: Temporary copy of assignments for trial
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    # temp_floors_by_space: Floors sorted by remaining area
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        # blk_area: Area of the current block
                        blk_area = blk['Cumulative_Area_SQM']
                        # blk_capacity: Capacity of the current block
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity and
                                can_place_block(blk, fl, temp_assignments, mode)):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        removed_blocks.append(trial_blocks.pop(0))

                for blk in removed_blocks:
                    # blk_area: Area of the removed block
                    blk_area = blk['Cumulative_Area_SQM']
                    # blk_capacity: Capacity of the removed block
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    # floors_by_space: Floors sorted by remaining area
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk, fl, assignments, mode)):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    # blk_area: Area of the block
                    blk_area = blk['Cumulative_Area_SQM']
                    # blk_capacity: Capacity of the block
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    # floors_by_space: Floors sorted by remaining area
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk, fl, assignments, mode)):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break
                    if not placed_block:
                        unassigned_blocks.append(blk)

    # ---------------------------------------
    # Phase 2: Assign Typical Blocks
    # ---------------------------------------

    # dept_unsplittable_groups: Dictionary of departments that cannot be split
    dept_unsplittable_groups = {}
    # splittable_blocks: List of blocks that can be split across floors
    splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        # dept: Department name
        dept = blk.get('Department_Sub_Department', '').strip()
        # spl: Splittable status
        spl = dept_splittable.get(dept, -1)
        if spl == 1:
            dept_unsplittable_groups.setdefault(dept, []).append(blk)
        else:
            splittable_blocks.append(blk)

    # Assign unsplittable department groups
    for dept, blocks_list in dept_unsplittable_groups.items():
        # total_a: Total area of the department's blocks
        total_a = sum(b.get('Cumulative_Area_SQM', 0) for b in blocks_list)
        # total_c: Total capacity of the department's blocks
        total_c = sum(b.get('Max_Occupancy_with_Capacity', 0) for b in blocks_list)
        placed = False

        # candidate_floors: Floors sorted by remaining area
        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= total_a and
                assignments[fl]['remaining_capacity'] >= total_c and
                all(can_place_block(b, fl, assignments, mode) for b in blocks_list)):
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    # cat: Space mix category
                    cat = primary_category(blk)
                    assignments[fl][cat + '_area'] += blk.get('Cumulative_Area_SQM', 0)
                    assignments[fl]['DeptArea'][dept] = assignments[fl]['DeptArea'].get(dept, 0) + blk.get('Cumulative_Area_SQM', 0)
                assignments[fl]['remaining_area'] -= total_a
                assignments[fl]['remaining_capacity'] -= total_c
                placed = True
                break

        if not placed:
            unassigned_blocks.extend(blocks_list)

    # Assign splittable blocks starting with ME category
    # me_blks: List of ME category blocks
    me_blks = [
        blk for blk in splittable_blocks
        if blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip() == 'ME'
    ]
    random.shuffle(me_blks)
    for blk in me_blks:
        # blk_area: Area of the block
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        # blk_capacity: Capacity of the block
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)
        # blk_dept: Department of the block
        blk_dept = blk.get('Department_Sub-Department', '').strip()

        # candidate_floors: Randomly shuffled floors
        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and
                assignments[fl]['remaining_capacity'] >= blk_capacity and
                can_place_block(blk, fl, assignments, mode)):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # Distribute remaining categories based on ME distribution
    # me_counts: Dictionary of ME block counts per floor
    me_counts = {fl: sum(1 for blk in assignments[fl]['assigned_blocks'] if primary_category(blk) == 'ME') for fl in floors}
    # tot_me: Total number of ME blocks
    tot_me = sum(me_counts.values()) or 1
    # frac_me: Fraction of ME blocks per floor
    frac_me = {f: me_counts[f] / tot_me for f in floors}

    for cat in ['WE', 'US', 'Support', 'Speciality']:
        # cat_blks: Blocks of the current category
        cat_blks = [b for b in splittable_blocks if primary_category(b) == cat]
        # total_cat: Total number of blocks in the category
        total_cat = len(cat_blks)
        if total_cat == 0:
            continue
        # raw_t: Target number of blocks per floor based on ME distribution
        raw_t = {f: frac_me[f] * total_cat for f in floors}
        # tgt: Rounded target number of blocks per floor
        tgt = {f: int(round(raw_t[f])) for f in floors}
        # diff: Difference between total target and actual blocks
        diff = total_cat - sum(tgt.values())
        if diff > 0:
            # frac_parts: Fractional parts for adjusting targets
            frac_parts = {f: raw_t[f] - math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x: frac_parts[x], reverse=True)[:diff]:
                tgt[f] += 1
        elif diff < 0:
            frac_parts = {f: raw_t[f] - math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x: frac_parts[x])[: -diff]:
                tgt[f] -= 1
        random.shuffle(cat_blks)
        # assigned: Tracks number of blocks assigned per floor
        assigned = {f: 0 for f in floors}
        for blk in cat_blks:
            # blk_area: Area of the block
            blk_area = blk.get('Cumulative_Area_SQM', 0)
            # blk_capacity: Capacity of the block
            blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)
            # blk_dept: Department of the block
            blk_dept = blk.get('Department_Sub-Department', '').strip()

            # deficits: Difference between target and assigned blocks per floor
            deficits = {f: tgt[f] - assigned[f] for f in floors}
            # cand: Candidate floors with positive deficits
            cand = [f for f in floors if deficits[f] > 0] or floors
            for fl in sorted(cand, key=lambda x: deficits.get(x, 0), reverse=True):
                if (assignments[fl]['remaining_area'] >= blk_area and
                    assignments[fl]['remaining_capacity'] >= blk_capacity and
                    can_place_block(blk, fl, assignments, mode)):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl][cat + '_area'] += blk_area
                    assigned[fl] += 1
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    break
            else:
                unassigned_blocks.append(blk)

    # ---------------------------------------
    # Phase 3: Attempt to Assign Unassigned Blocks
    # ---------------------------------------
    # still_unassigned: List of blocks that remain unassigned after retry
    still_unassigned = []
    for blk in unassigned_blocks:
        # blk_area: Area of the block
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        # blk_capacity: Capacity of the block
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)
        placed = False
        # floors_by_space: Floors sorted by remaining area
        floors_by_space = sorted(floors, key=lambda x: assignments[x]['remaining_area'], reverse=True)
        for fl in floors_by_space:
            if (assignments[fl]['remaining_area'] >= blk_area and
                assignments[fl]['remaining_capacity'] >= blk_capacity and
                can_place_block(blk, fl, assignments, mode)):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
                # cat: Space mix category
                cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
                if cat == 'ME':
                    assignments[fl]['ME_area'] += blk_area
                elif cat == 'WE':
                    assignments[fl]['WE_area'] += blk_area
                elif cat == 'US':
                    assignments[fl]['US_area'] += blk_area
                elif cat.lower() == 'support':
                    assignments[fl]['Support_area'] += blk_area
                elif cat.lower() == 'speciality':
                    assignments[fl]['Speciality_area'] += blk_area
                placed = True
                break
        if not placed:
            still_unassigned.append(blk)

    unassigned_blocks = still_unassigned

    # ---------------------------------------
    # Generate Output DataFrames
    # ---------------------------------------
    # rows: List of dictionaries for detailed assignment DataFrame
    rows = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            rows.append({
                'Floor': fl,
                'Department': blk.get('Department_Sub_Department', ''),
                'Block_Name': blk.get('Block_Name', ''),
                'Destination_Group': blk.get('Destination_Group', ''),
                'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
                'Assigned_Area_SQM': blk.get('Cumulative_Area_SQM', 0),
                'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
                'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '')
            })
    # detailed_df: DataFrame of detailed block assignments
    detailed_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
        'Floor', 'Department', 'Block_Name', 'Destination_Group', 'SpaceMix',
        'Assigned_Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'
    ])

    # summary: DataFrame summarizing assignments by floor
    summary = (
        detailed_df.groupby('Floor')
        .agg(
            Assgn_Blocks=('Block_Name', 'count'),
            Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
            Total_Occupancy=('Max_Occupancy', 'sum')
        )
        .reset_index()
    ) if not detailed_df.empty else pd.DataFrame(columns=['Floor', 'Assgn_Blocks', 'Assgn_Area_SQM', 'Total_Occupancy'])

    # input_sub: Subset of floor data for merging
    input_sub = all_floor_data[['Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity']].rename(
        columns={
            'Name': 'Floor',
            'Usable_Area_(SQM)': 'Input_Usable_Area_SQM',
            'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
        }
    )
    # floor_summary_df: Merged summary of floor assignments
    floor_summary_df = pd.merge(input_sub, summary, on='Floor', how='left').fillna(0)

    # ---------------------------------------
    # Space Mix Distribution
    # ---------------------------------------
    # category_totals: Dictionary of total blocks per category
    category_totals = {
        cat: len(all_block_data[
            all_block_data['SpaceMix_(ME_WE_US_Support_Speciality)'].astype(str).str.strip() == cat
        ])
        for cat in all_categories
    }

    # mix_rows: List of dictionaries for space mix DataFrame
    mix_rows = []
    for fl, info in assignments.items():
        # counts: Dictionary of block counts per category on this floor
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            # cat: Space mix category
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
            if cat in counts:
                counts[cat] += 1
        # total_blocks_on_floor: Total blocks on this floor
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            # cnt: Number of blocks of this category
            cnt = counts[cat]
            # pct_of_floor: Percentage of floor's blocks for this category
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # total_cat: Total blocks in this category overall
            total_cat = category_totals.get(cat, 0)
            # pct_overall: Percentage of overall blocks for this category
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            mix_rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)
            })

    # space_mix_df: DataFrame of space mix percentages
    space_mix_df = pd.DataFrame(mix_rows) if mix_rows else pd.DataFrame(columns=['Floor', 'SpaceMix', '%spaceMix'])

    # ---------------------------------------
    # Unassigned Blocks DataFrame
    # ---------------------------------------
    # unassigned_list: List of dictionaries for unassigned blocks
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '')
        })
    # unassigned_df: DataFrame of unassigned blocks
    unassigned_df = pd.DataFrame(unassigned_list) if unassigned_list else pd.DataFrame(columns=[
        'Department', 'Block_Name', 'Destination_Group', 'SpaceMix', 'Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'
    ])

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ---------------------------------------
# Step 5: Generate and Export Outputs
# ---------------------------------------

# Run stacking plan for each mode
# central_detailed, central_floor_sum, central_space_mix, central_unassigned: Results for centralized mode
central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
# semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned: Results for semi-centralized mode
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = run_stack_plan('semi')
# decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned: Results for decentralized mode
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')

# Define output file names
# central_file: Output Excel file for centralized mode
central_file = 'AAR1_stack_plan_centralized14.xlsx'
# semi_file: Output Excel file for semi-centralized mode
semi_file = 'AAR1_stack_plan_semi_centralized14.xlsx'
# decentral_file: Output Excel file for decentralized mode
decentral_file = 'AAR1_stack_plan_decentralized14.xlsx'

# Export results to Excel
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

# Print confirmation of output generation
print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")

# AAR2

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets & Normalize
# ----------------------------------------

excel_path = '/content/AA- R2 (1).xlsx'  # ← adjust if needed

# 1.1 Floors sheet (skip first row)
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=1  # skips the first row (0-indexed)
)

all_floor_data.columns = all_floor_data.columns.str.strip()

# Normalize usable-area & capacity column names
floor_col_map = {}
for c in all_floor_data.columns:
    key = c.lower().replace(' ', '').replace('_','')
    if 'usable' in key and 'area' in key:
        floor_col_map[c] = 'Usable_Area'
    elif 'capacity' in key or 'loading' in key:
        floor_col_map[c] = 'Max_Assignable_Floor_loading_Capacity'
all_floor_data = all_floor_data.rename(columns=floor_col_map)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Renovation Program Table Input '
)
all_block_data.columns = all_block_data.columns.str.strip()

# —————————————————————————————————————————————
# Peel off Immovable Assets if those columns exist
# —————————————————————————————————————————————
if {'Immovable-Movable Asset', 'Level'}.issubset(all_block_data.columns):
    immovable_df = all_block_data.loc[
        all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
    ].copy()
    immovable_df['Assigned_Floor'] = immovable_df['Level'].astype(str).str.strip()
    movable_blocks_df = all_block_data.drop(immovable_df.index).copy()
else:
    immovable_df = pd.DataFrame(columns=list(all_block_data.columns) + ['Assigned_Floor'])
    movable_blocks_df = all_block_data.copy()

# 1.3 Department Split sheet
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)

# 1.4 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [n for n in xls.sheet_names if "Adjacency" in n][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.5 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current = None
for _, r in df_logic.iterrows():
    cell = str(r[0]).strip() if pd.notna(r[0]) else ""
    if cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current = cell
        De_Centralized_data[current] = {"Add": 0}
    elif current and cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current]["Add"] = int(r[1]) if pd.notna(r[1]) else 0
for k in ["Centralised", "Semi Centralized", "DeCentralised"]:
    De_Centralized_data.setdefault(k, {"Add": 0})
# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Convert cumulative circulation area from SQFT → SQM
#all_block_data['Cumulative_Area_SQM'] = (
 #   all_block_data['Cumulative_Block_Circulation_Area_(SQM)']
#)

# Step 0: Assume all_block_data is already defined as your full DataFrame

# Step 1: Select Immovable Asset blocks
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
].copy()

# Step 2: Select all other blocks (i.e., NOT immovable → movable or NA or others)
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() != 'Immovable Asset'
].copy()


# Step 2.1: From Movable blocks, select Destination or both
destination_blocks = movable_blocks[movable_blocks['Typical_Destination'].isin(['Destination', 'both'])].copy()

# Step 2.2: From Movable blocks, select Typical
typical_blocks = movable_blocks[movable_blocks['Typical_Destination'] == 'Typical'].copy()


# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry tracks:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      1) detailed_df      – each block’s assigned floor, department, block name, destination group, space mix, area, occupancy
      2) floor_summary_df – floor‐wise totals (block count, total area, total occupancy)
      3) space_mix_df     – for each floor and each category {ME, WE, US, Support, Speciality}:
                              - Unit_Count_on_Floor
                              - Pct_of_Floor_UC      = (category_count_on_floor / total_blocks_on_floor) × 100%
                              - Pct_of_Overall_UC    = (category_count_on_floor / total_blocks_of_category_overall) × 100%
      4) unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    import re

    def normalize_floor_name(name):
        """Standardize floor names like 'L002Floor 01' or '3 Floor' → 'Floor 03'"""
        name = str(name).strip()

        # Remove prefix like L001
        name = re.sub(r'^L\d{3}', '', name).strip()

        # If format is like '3 Floor' → convert to 'Floor 03'
        match = re.match(r'^(\d+)\s+Floor$', name)
        if match:
            number = int(match.group(1))
            return f"Floor {number:02d}"

        return name  # for 'Ground Floor', 'Floor 01', etc.
      # Map normalized name → actual floor name in assignments
    floor_name_map = {
        normalize_floor_name(row['Name']): row['Name'].strip()
        for _, row in all_floor_data.iterrows()
    }





    # Assign immovable blocks based on 'Level'
    immovable_blocks = all_block_data[all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'].copy()

    for _, block in immovable_blocks.iterrows():
        target_floor_raw = str(block['Level']).strip()
        target_floor = floor_name_map.get(target_floor_raw, None)

        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']

        if target_floor in assignments:
            floor_data = assignments[target_floor]

            # Check area and capacity constraints
            if floor_data['remaining_area'] >= block_area and floor_data['remaining_capacity'] >= block_capacity:
                # Assign block
                floor_data['assigned_blocks'].append(block.to_dict())
                floor_data['assigned_departments'].add(block['Department_Sub_Department'])

                # Update remaining area and capacity
                floor_data['remaining_area'] -= block_area
                floor_data['remaining_capacity'] -= block_capacity

                # Update area category
                category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
                category_key = f"{category}_area"
                if category_key in floor_data:
                    floor_data[category_key] += block_area
            else:
                unassigned_blocks.append(block.to_dict())
        else:
            unassigned_blocks.append(block.to_dict())


    # Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = destination_floor_count()
    # Cap at total number of floors
    max_dest_floors = min(max_dest_floors, len(floors))

    # Pre‐compute each group's total area and total capacity
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Block_Circulation_Area']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Block_Circulation_Area'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Block_Circulation_Area'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Block_Circulation_Area']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Block_Circulation_Area']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= bll_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)

    # Phase 2A: Randomly assign ME blocks (typical)
    me_blocks = [
        blk for blk in typical_blocks.to_dict('records')
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Block_Circulation_Area']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            print(f"Warning: Could not place ME block '{blk['Block_Name']}'")

    # Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {fl: me_count_per_floor[fl] / total_me for fl in floors}

    # Phase 2B: Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in typical_blocks.to_dict('records')
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Block_Circulation_Area']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                print(f"Warning: Could not place {category} block '{blk['Block_Name']}'")
                unassigned_blocks.append(blk)
    # Re-attempt placing unassigned blocks on randomized floor order
    still_unassigned = []

    # Get list of floor names and shuffle
    floor_list = list(assignments.keys())
    random.shuffle(floor_list)  # This randomizes the order

    for block in unassigned_blocks:
        placed = False
        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']
        department = block['Department_Sub_Department']
        category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
        category_key = f"{category}_area"

        for floor in floor_list:
            data = assignments[floor]
            if data['remaining_area'] >= block_area and data['remaining_capacity'] >= block_capacity:
                # Assign the block to this floor
                data['assigned_blocks'].append(block)
                data['assigned_departments'].add(department)
                data['remaining_area'] -= block_area
                data['remaining_capacity'] -= block_capacity

                # Update category area
                if category_key in data:
                    data[category_key] += block_area

                placed = True
                break  # Move to next block

        if not placed:
            still_unassigned.append(block)

    # Update the global unassigned_blocks list
    unassigned_blocks = still_unassigned

    # Phase 3: Build Detailed & Summary DataFrames

    # 3.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Block_id': blk['Block_ID'],
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Block_Circulation_Area'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity'],
                'Asset_Type': blk['Immovable-Movable Asset']  # <-- New column added
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area': 'Input_Usable_Area',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)
    # 3.3 SpaceMix_By_Units DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                'Unit_Count_on_Floor': cnt,
                'Pct_of_Floor_UC': round(pct_of_floor, 2),
                'Pct_of_Overall_UC': round(pct_overall, 2)
            })

    space_mix_df = pd.DataFrame(rows)

     # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Asset_Type': blk['Immovable-Movable Asset']
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Three Excel Files
# ----------------------------------------
central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')


with pd.ExcelWriter('stack_plan_centralized20.xlsx') as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_semi_centralized20.xlsx') as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_decentralized20.xlsx') as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


print("✅ Generated three Excel outputs:")
print("    • stack_plan_centralized8.xlsx")
print("    • stack_plan_semi_centralized8.xlsx")
print("    • stack_plan_decentralized8.xlsx")